In [ ]:
import os
import sys
import gc
import pandas as pd
import pandas_ta as ta
from math import pi
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.transform import factor_cmap
from bokeh.palettes import Spectral6
import joblib
from scipy.stats import linregress
from bokeh.resources import INLINE
import bokeh.io

In [ ]:
bokeh.io.output_notebook(INLINE)

In [ ]:
TEST_MINIMUM_DATE_DATAFRAME = 20211001
TEST_MINIMUM_DATE_TRADE = 20211029

TEST_MAX_TRAIN_DATE = 20220731

TEST_CURRENT_DAILY_FILE_CSV = 'WDO$Daily_test.csv'
TEST_CURRENT_5MIN_FILE_CSV = 'WDO$M5_test.csv'

DECISION_BOUNDARY = 0.4

In [ ]:
data_folder = os.path.join('..','00.data','output')

files_found = [x for x in os.listdir(data_folder) if x.endswith('.pickle')]
files_found_tokens=[x.split('.') for x in files_found]

models_found = {}

for current_model in files_found_tokens:
    model_name = '.'.join(current_model[:-2])
    filename = os.path.join('..','00.data','output','.'.join(current_model))
    if not model_name in models_found:
        models_found[model_name]= {}          
    models_found[model_name][current_model[-2]]=filename

In [ ]:
first_model = list(models_found.keys())[0]
current_parameters = joblib.load(models_found[first_model]['parameters'])
MINIMUM_TIME = current_parameters['MINIMUM_TIME']
MAXIMUM_TIME = current_parameters['MAXIMUM_TIME']
CURRENT_ASSET = current_parameters['CURRENT_ASSET']
CURRENT_TIMEFRAME= current_parameters['CURRENT_TIMEFRAME']
CURRENT_TARGET= current_parameters['CURRENT_TARGET']
CURRENT_STOP= current_parameters['CURRENT_STOP']
MAX_TRADE_DURATION = current_parameters['MAX_TRADE_DURATION']


In [ ]:
csv_diario = pd.read_csv(os.path.join('..','00.data','input',TEST_CURRENT_DAILY_FILE_CSV))
csv_5Min = pd.read_csv(os.path.join('..','00.data','input',TEST_CURRENT_5MIN_FILE_CSV))

csv_diario['Date'] = csv_diario.apply(lambda x: int(x['datetime'].replace('.','')), axis=1)
csv_5Min['Date'] = csv_5Min.apply(lambda x: int(x['datetime'].split(' ')[0].replace('.','')), axis=1)
csv_5Min['Time'] = csv_5Min.apply(lambda x: int(x['datetime'].split(' ')[1].replace(':','')), axis=1)

csv_diario = csv_diario[csv_diario['Date'] >= TEST_MINIMUM_DATE_DATAFRAME]
csv_5Min = csv_5Min[csv_5Min['Date'] >= TEST_MINIMUM_DATE_DATAFRAME]

csv_5Min['ema9'] = csv_5Min.ta.ema(length=9)
csv_5Min['ema21'] = csv_5Min.ta.ema(length=21)
csv_5Min['ema55'] = csv_5Min.ta.ema(length=55)
csv_5Min['ema144'] = csv_5Min.ta.ema(length=144)
csv_5Min['ema233'] = csv_5Min.ta.ema(length=233)

In [ ]:
data_daily = csv_diario.to_dict('records')
data_5min = csv_5Min.to_dict('records')
daily_dates = []

ohlc_daily = {}

for current_date in data_daily:
    ohlc_daily[current_date['Date']] = current_date
    daily_dates.append(current_date['Date'])

In [ ]:
def get_slope(current_bar, value_to_calculate):
    a = []
    b = []
    for current_x in range(0,11):
        a.append(current_x)
        b.append(current_bar[f'x{current_x}_{value_to_calculate}'])
    return linregress(a,b).slope

In [ ]:
generated_bars = []
generated_bars_raw = []
last_current_date = ''
current_bar_in_date = 0


for index, _ in enumerate(data_5min):
    current_date = data_5min[index]['Date']
    current_time = data_5min[index]['Time']
    
    if last_current_date != current_date:
        last_current_date = current_date
        current_bar_in_date = 0
    else:
        current_bar_in_date += 1
    
    
    raw_bar = {}
    for x in ['open','high','low','close','volume','Date','Time']:
        raw_bar[x] = data_5min[index][x]
    
    generated_bars_raw.append(raw_bar)
    
    if current_time < MINIMUM_TIME or \
        current_time > MAXIMUM_TIME or \
        current_date < TEST_MINIMUM_DATE_TRADE:
        continue
    
    current_open = data_5min[index]['open']
    current_volume = data_5min[index]['volume']
    
    previous_date = [x for x in daily_dates if x < current_date]
    previous_date = previous_date[len(previous_date)-1]
    previous_date = ohlc_daily[previous_date]
    
    current_bar = {}
    
    current_bar['current_open'] = current_open
    current_bar['current_date'] = current_date
    current_bar['current_time'] = current_time
    current_bar['current_bar_in_date']= 0
    
    current_bar['previous_date'] = previous_date['Date']
    current_bar['previous_high_real'] = previous_date['high']
    current_bar['previous_close_real'] = previous_date['close']
    current_bar['previous_low_real'] = previous_date['low']
    current_bar['previous_open_real'] = previous_date['open']    
    current_bar['previous_high'] = ((previous_date['high'] / current_open) - 1) * 100
    current_bar['previous_close'] = ((previous_date['close'] / current_open) - 1) * 100
    current_bar['previous_low'] = ((previous_date['low'] / current_open) - 1) * 100
    current_bar['previous_open'] =((previous_date['open'] / current_open) - 1) * 100
    
    current_bar['x0_ema9'] = ((data_5min[index-11]['ema9'] / current_open) - 1) * 100
    current_bar['x0_ema21'] = ((data_5min[index-11]['ema21'] / current_open) - 1) * 100
    current_bar['x0_ema55'] = ((data_5min[index-11]['ema55'] / current_open) - 1) * 100
    current_bar['x0_ema144'] = ((data_5min[index-11]['ema144'] / current_open) - 1) * 100
    current_bar['x0_ema233'] = ((data_5min[index-11]['ema233'] / current_open) - 1) * 100    
    current_bar['x0_ema9_real'] = data_5min[index-11]['ema9'] 
    current_bar['x0_ema21_real'] = data_5min[index-11]['ema21'] 
    current_bar['x0_ema55_real'] = data_5min[index-11]['ema55'] 
    current_bar['x0_ema144_real'] = data_5min[index-11]['ema144'] 
    current_bar['x0_ema233_real'] = data_5min[index-11]['ema233'] 
    current_bar['x0_open'] = ((data_5min[index-11]['open'] / current_open) - 1) * 100
    current_bar['x0_high'] = ((data_5min[index-11]['high'] / current_open) - 1) * 100
    current_bar['x0_low'] = ((data_5min[index-11]['low'] / current_open) - 1) * 100
    current_bar['x0_close'] = ((data_5min[index-11]['close'] / current_open) - 1) * 100
    current_bar['x0_volume'] = ((data_5min[index-11]['volume'] / current_volume) - 1) * 100
    current_bar['x0_open_real'] = data_5min[index-11]['open'] 
    current_bar['x0_high_real'] = data_5min[index-11]['high'] 
    current_bar['x0_low_real'] = data_5min[index-11]['low'] 
    current_bar['x0_close_real'] = data_5min[index-11]['close'] 
    current_bar['x0_height'] = current_bar['x0_high'] - current_bar['x0_low']
    
    current_bar['x0_body'] = max([current_bar['x0_open_real'],  current_bar['x0_close_real']]) / min([current_bar['x0_open_real'],  current_bar['x0_close_real']]) * 100
    current_bar['x0_roc'] = ((current_bar['x0_open_real'] / current_bar['x0_close_real']) - 1) * 100
    current_bar['x0_previous_high'] = ((current_bar['previous_high_real'] / current_bar['x0_close_real']) -1) * 100
    current_bar['x0_previous_low'] = ((current_bar['previous_low_real'] / current_bar['x0_close_real']) -1) * 100
    current_bar['x0_previous_close'] = ((current_bar['previous_close_real'] / current_bar['x0_close_real']) -1) * 100
    current_bar['x0_previous_open'] = ((current_bar['previous_open_real'] / current_bar['x0_close_real']) -1) * 100
    current_bar['x0_ema9_close'] = ((current_bar['x0_ema9_real'] / current_bar['x0_close_real']) -1) * 100
    current_bar['x0_ema21_close'] = ((current_bar['x0_ema21_real'] / current_bar['x0_close_real']) -1) * 100
    current_bar['x0_ema55_close'] = ((current_bar['x0_ema55_real'] / current_bar['x0_close_real']) -1) * 100
    current_bar['x0_ema144_close'] = ((current_bar['x0_ema144_real'] / current_bar['x0_close_real']) -1) * 100
    current_bar['x0_ema233_close'] = ((current_bar['x0_ema233_real'] / current_bar['x0_close_real']) -1) * 100
    
    
    current_bar['x1_ema9'] = ((data_5min[index-10]['ema9'] / current_open) - 1) * 100
    current_bar['x1_ema21'] = ((data_5min[index-10]['ema21'] / current_open) - 1) * 100
    current_bar['x1_ema55'] = ((data_5min[index-10]['ema55'] / current_open) - 1) * 100
    current_bar['x1_ema144'] = ((data_5min[index-10]['ema144'] / current_open) - 1) * 100
    current_bar['x1_ema233'] = ((data_5min[index-10]['ema233'] / current_open) - 1) * 100        
    current_bar['x1_ema9_real'] = data_5min[index-10]['ema9'] 
    current_bar['x1_ema21_real'] = data_5min[index-10]['ema21']
    current_bar['x1_ema55_real'] = data_5min[index-10]['ema55'] 
    current_bar['x1_ema144_real'] = data_5min[index-10]['ema144'] 
    current_bar['x1_ema233_real'] = data_5min[index-10]['ema233'] 
    current_bar['x1_open'] = ((data_5min[index-10]['open'] / current_open) - 1) * 100
    current_bar['x1_high'] = ((data_5min[index-10]['high'] / current_open) - 1) * 100
    current_bar['x1_low'] = ((data_5min[index-10]['low'] / current_open) - 1) * 100
    current_bar['x1_close'] = ((data_5min[index-10]['close'] / current_open) - 1) * 100
    current_bar['x1_volume'] = ((data_5min[index-10]['volume'] / current_volume) - 1) * 100
    current_bar['x1_open_real'] = data_5min[index-10]['open'] 
    current_bar['x1_high_real'] = data_5min[index-10]['high'] 
    current_bar['x1_low_real'] = data_5min[index-10]['low'] 
    current_bar['x1_close_real'] = data_5min[index-10]['close'] 
    current_bar['x1_height'] = current_bar['x1_high'] - current_bar['x1_low']
    current_bar['x1_body'] = max([current_bar['x1_open_real'],  current_bar['x1_close_real']]) / min([current_bar['x1_open_real'],  current_bar['x1_close_real']])  * 100

    current_bar['x1_roc'] = ((current_bar['x1_open_real'] / current_bar['x1_close_real']) - 1) * 100
    current_bar['x1_previous_high'] = ((current_bar['previous_high_real'] / current_bar['x1_close_real']) -1) * 100
    current_bar['x1_previous_low'] = ((current_bar['previous_low_real'] / current_bar['x1_close_real']) -1) * 100
    current_bar['x1_previous_close'] = ((current_bar['previous_close_real'] / current_bar['x1_close_real']) -1) * 100
    current_bar['x1_previous_open'] = ((current_bar['previous_open_real'] / current_bar['x1_close_real']) -1) * 100
    current_bar['x1_ema9_close'] = ((current_bar['x1_ema9_real'] / current_bar['x1_close_real']) -1) * 100
    current_bar['x1_ema21_close'] = ((current_bar['x1_ema21_real'] / current_bar['x1_close_real']) -1) * 100
    current_bar['x1_ema55_close'] = ((current_bar['x1_ema55_real'] / current_bar['x1_close_real']) -1) * 100
    current_bar['x1_ema144_close'] = ((current_bar['x1_ema144_real'] / current_bar['x1_close_real']) -1) * 100
    current_bar['x1_ema233_close'] = ((current_bar['x1_ema233_real'] / current_bar['x1_close_real']) -1) * 100
    current_bar['x1_x0_close'] = ((current_bar['x1_close_real'] / current_bar['x0_close_real']) -1) * 100
    current_bar['x1_x0_high'] = ((current_bar['x1_high_real'] / current_bar['x0_high_real']) -1) * 100
    current_bar['x1_x0_low'] = ((current_bar['x1_low_real'] / current_bar['x0_low_real']) -1) * 100
    
    
    current_bar['x2_ema9'] = ((data_5min[index-9]['ema9'] / current_open) - 1) * 100
    current_bar['x2_ema21'] = ((data_5min[index-9]['ema21'] / current_open) - 1) * 100
    current_bar['x2_ema55'] = ((data_5min[index-9]['ema55'] / current_open) - 1) * 100
    current_bar['x2_ema144'] = ((data_5min[index-9]['ema144'] / current_open) - 1) * 100
    current_bar['x2_ema233'] = ((data_5min[index-9]['ema233'] / current_open) - 1) * 100        
    current_bar['x2_ema9_real'] = data_5min[index-9]['ema9'] 
    current_bar['x2_ema21_real'] = data_5min[index-9]['ema21'] 
    current_bar['x2_ema55_real'] = data_5min[index-9]['ema55'] 
    current_bar['x2_ema144_real'] = data_5min[index-9]['ema144']
    current_bar['x2_ema233_real'] = data_5min[index-9]['ema233']
    current_bar['x2_open'] = ((data_5min[index-9]['open'] / current_open) - 1) * 100
    current_bar['x2_high'] = ((data_5min[index-9]['high'] / current_open) - 1) * 100
    current_bar['x2_low'] = ((data_5min[index-9]['low'] / current_open) - 1) * 100
    current_bar['x2_close'] = ((data_5min[index-9]['close'] / current_open) - 1) * 100
    current_bar['x2_volume'] = ((data_5min[index-9]['volume'] / current_volume) - 1) * 100    
    current_bar['x2_open_real'] = data_5min[index-9]['open'] 
    current_bar['x2_high_real'] = data_5min[index-9]['high'] 
    current_bar['x2_low_real'] = data_5min[index-9]['low'] 
    current_bar['x2_close_real'] = data_5min[index-9]['close']
    current_bar['x2_height'] = current_bar['x2_high'] - current_bar['x2_low']
    current_bar['x2_body'] = max([current_bar['x2_open_real'],  current_bar['x2_close_real']]) / min([current_bar['x2_open_real'],  current_bar['x2_close_real']]) * 100

    current_bar['x2_roc'] = ((current_bar['x2_open_real'] / current_bar['x2_close_real']) - 1) * 100
    current_bar['x2_previous_high'] = ((current_bar['previous_high_real'] / current_bar['x2_close_real']) -1) * 100
    current_bar['x2_previous_low'] = ((current_bar['previous_low_real'] / current_bar['x2_close_real']) -1) * 100
    current_bar['x2_previous_close'] = ((current_bar['previous_close_real'] / current_bar['x2_close_real']) -1) * 100
    current_bar['x2_previous_open'] = ((current_bar['previous_open_real'] / current_bar['x2_close_real']) -1) * 100
    current_bar['x2_ema9_close'] = ((current_bar['x2_ema9_real'] / current_bar['x2_close_real']) -1) * 100
    current_bar['x2_ema21_close'] = ((current_bar['x2_ema21_real'] / current_bar['x2_close_real']) -1) * 100
    current_bar['x2_ema55_close'] = ((current_bar['x2_ema55_real'] / current_bar['x2_close_real']) -1) * 100
    current_bar['x2_ema144_close'] = ((current_bar['x2_ema144_real'] / current_bar['x2_close_real']) -1) * 100
    current_bar['x2_ema233_close'] = ((current_bar['x2_ema233_real'] / current_bar['x2_close_real']) -1) * 100
    current_bar['x2_x1_close'] = ((current_bar['x2_close_real'] / current_bar['x1_close_real']) -1) * 100
    current_bar['x2_x1_high'] = ((current_bar['x2_high_real'] / current_bar['x1_high_real']) -1) * 100
    current_bar['x2_x1_low'] = ((current_bar['x2_low_real'] / current_bar['x1_low_real']) -1) * 100    
    

    
    current_bar['x3_ema9'] = ((data_5min[index-8]['ema9'] / current_open) - 1) * 100
    current_bar['x3_ema21'] = ((data_5min[index-8]['ema21'] / current_open) - 1) * 100
    current_bar['x3_ema55'] = ((data_5min[index-8]['ema55'] / current_open) - 1) * 100
    current_bar['x3_ema144'] = ((data_5min[index-8]['ema144'] / current_open) - 1) * 100
    current_bar['x3_ema233'] = ((data_5min[index-8]['ema233'] / current_open) - 1) * 100        
    current_bar['x3_ema9_real'] = data_5min[index-8]['ema9'] 
    current_bar['x3_ema21_real'] = data_5min[index-8]['ema21'] 
    current_bar['x3_ema55_real'] = data_5min[index-8]['ema55'] 
    current_bar['x3_ema144_real'] = data_5min[index-8]['ema144'] 
    current_bar['x3_ema233_real'] = data_5min[index-8]['ema233'] 
    current_bar['x3_open'] = ((data_5min[index-8]['open'] / current_open) - 1) * 100
    current_bar['x3_high'] = ((data_5min[index-8]['high'] / current_open) - 1) * 100
    current_bar['x3_low'] = ((data_5min[index-8]['low'] / current_open) - 1) * 100
    current_bar['x3_close'] = ((data_5min[index-8]['close'] / current_open) - 1) * 100
    current_bar['x3_volume'] = ((data_5min[index-8]['volume'] / current_volume) - 1) * 100        
    current_bar['x3_open_real'] = data_5min[index-8]['open'] 
    current_bar['x3_high_real'] = data_5min[index-8]['high'] 
    current_bar['x3_low_real'] = data_5min[index-8]['low'] 
    current_bar['x3_close_real'] = data_5min[index-8]['close']
    current_bar['x3_height'] = current_bar['x3_high'] - current_bar['x3_low']
    current_bar['x3_body'] = max([current_bar['x3_open_real'],  current_bar['x3_close_real']]) / min([current_bar['x3_open_real'],  current_bar['x3_close_real']])  * 100    
    
    current_bar['x3_roc'] = ((current_bar['x3_open_real'] / current_bar['x3_close_real']) - 1) * 100
    current_bar['x3_previous_high'] = ((current_bar['previous_high_real'] / current_bar['x3_close_real']) -1) * 100
    current_bar['x3_previous_low'] = ((current_bar['previous_low_real'] / current_bar['x3_close_real']) -1) * 100
    current_bar['x3_previous_close'] = ((current_bar['previous_close_real'] / current_bar['x3_close_real']) -1) * 100
    current_bar['x3_previous_open'] = ((current_bar['previous_open_real'] / current_bar['x3_close_real']) -1) * 100
    current_bar['x3_ema9_close'] = ((current_bar['x3_ema9_real'] / current_bar['x3_close_real']) -1) * 100
    current_bar['x3_ema21_close'] = ((current_bar['x3_ema21_real'] / current_bar['x3_close_real']) -1) * 100
    current_bar['x3_ema55_close'] = ((current_bar['x3_ema55_real'] / current_bar['x3_close_real']) -1) * 100
    current_bar['x3_ema144_close'] = ((current_bar['x3_ema144_real'] / current_bar['x3_close_real']) -1) * 100
    current_bar['x3_ema233_close'] = ((current_bar['x3_ema233_real'] / current_bar['x3_close_real']) -1) * 100
    current_bar['x3_x2_close'] = ((current_bar['x3_close_real'] / current_bar['x2_close_real']) -1) * 100
    current_bar['x3_x2_high'] = ((current_bar['x3_high_real'] / current_bar['x2_high_real']) -1) * 100
    current_bar['x3_x2_low'] = ((current_bar['x3_low_real'] / current_bar['x2_low_real']) -1) * 100    
    
    
    
    current_bar['x4_ema9'] = ((data_5min[index-7]['ema9'] / current_open) - 1) * 100
    current_bar['x4_ema21'] = ((data_5min[index-7]['ema21'] / current_open) - 1) * 100
    current_bar['x4_ema55'] = ((data_5min[index-7]['ema55'] / current_open) - 1) * 100
    current_bar['x4_ema144'] = ((data_5min[index-7]['ema144'] / current_open) - 1) * 100
    current_bar['x4_ema233'] = ((data_5min[index-7]['ema233'] / current_open) - 1) * 100        
    current_bar['x4_ema9_real'] = data_5min[index-7]['ema9'] 
    current_bar['x4_ema21_real'] = data_5min[index-7]['ema21']
    current_bar['x4_ema55_real'] = data_5min[index-7]['ema55']
    current_bar['x4_ema144_real'] = data_5min[index-7]['ema144']
    current_bar['x4_ema233_real'] = data_5min[index-7]['ema233']
    current_bar['x4_open'] = ((data_5min[index-7]['open'] / current_open) - 1) * 100
    current_bar['x4_high'] = ((data_5min[index-7]['high'] / current_open) - 1) * 100
    current_bar['x4_low'] = ((data_5min[index-7]['low'] / current_open) - 1) * 100
    current_bar['x4_close'] = ((data_5min[index-7]['close'] / current_open) - 1) * 100
    current_bar['x4_volume'] = ((data_5min[index-7]['volume'] / current_volume) - 1) * 100            
    current_bar['x4_open_real'] = data_5min[index-7]['open'] 
    current_bar['x4_high_real'] = data_5min[index-7]['high'] 
    current_bar['x4_low_real'] = data_5min[index-7]['low'] 
    current_bar['x4_close_real'] = data_5min[index-7]['close'] 
    current_bar['x4_height'] = current_bar['x4_high'] - current_bar['x4_low']
    current_bar['x4_body'] = max([current_bar['x4_open_real'],  current_bar['x4_close_real']]) / min([current_bar['x4_open_real'],  current_bar['x4_close_real']]) * 100

    current_bar['x4_roc'] = ((current_bar['x4_open_real'] / current_bar['x4_close_real']) - 1) * 100
    current_bar['x4_previous_high'] = ((current_bar['previous_high_real'] / current_bar['x4_close_real']) -1) * 100
    current_bar['x4_previous_low'] = ((current_bar['previous_low_real'] / current_bar['x4_close_real']) -1) * 100
    current_bar['x4_previous_close'] = ((current_bar['previous_close_real'] / current_bar['x4_close_real']) -1) * 100
    current_bar['x4_previous_open'] = ((current_bar['previous_open_real'] / current_bar['x4_close_real']) -1) * 100
    current_bar['x4_ema9_close'] = ((current_bar['x4_ema9_real'] / current_bar['x4_close_real']) -1) * 100
    current_bar['x4_ema21_close'] = ((current_bar['x4_ema21_real'] / current_bar['x4_close_real']) -1) * 100
    current_bar['x4_ema55_close'] = ((current_bar['x4_ema55_real'] / current_bar['x4_close_real']) -1) * 100
    current_bar['x4_ema144_close'] = ((current_bar['x4_ema144_real'] / current_bar['x4_close_real']) -1) * 100
    current_bar['x4_ema233_close'] = ((current_bar['x4_ema233_real'] / current_bar['x4_close_real']) -1) * 100
    current_bar['x4_x3_close'] = ((current_bar['x4_close_real'] / current_bar['x3_close_real']) -1) * 100
    current_bar['x4_x3_high'] = ((current_bar['x4_high_real'] / current_bar['x3_high_real']) -1) * 100
    current_bar['x4_x3_low'] = ((current_bar['x4_low_real'] / current_bar['x3_low_real']) -1) * 100        
    
    current_bar['x5_ema9'] = ((data_5min[index-6]['ema9'] / current_open) - 1) * 100
    current_bar['x5_ema21'] = ((data_5min[index-6]['ema21'] / current_open) - 1) * 100
    current_bar['x5_ema55'] = ((data_5min[index-6]['ema55'] / current_open) - 1) * 100
    current_bar['x5_ema144'] = ((data_5min[index-6]['ema144'] / current_open) - 1) * 100
    current_bar['x5_ema233'] = ((data_5min[index-6]['ema233'] / current_open) - 1) * 100        
    current_bar['x5_ema9_real'] = data_5min[index-6]['ema9'] 
    current_bar['x5_ema21_real'] = data_5min[index-6]['ema21'] 
    current_bar['x5_ema55_real'] = data_5min[index-6]['ema55'] 
    current_bar['x5_ema144_real'] = data_5min[index-6]['ema144']
    current_bar['x5_ema233_real'] = data_5min[index-6]['ema233']
    current_bar['x5_open'] = ((data_5min[index-6]['open'] / current_open) - 1) * 100
    current_bar['x5_high'] = ((data_5min[index-6]['high'] / current_open) - 1) * 100
    current_bar['x5_low'] = ((data_5min[index-6]['low'] / current_open) - 1) * 100
    current_bar['x5_close'] = ((data_5min[index-6]['close'] / current_open) - 1) * 100
    current_bar['x5_volume'] = ((data_5min[index-6]['volume'] / current_volume) - 1) * 100            
    current_bar['x5_open_real'] = data_5min[index-6]['open'] 
    current_bar['x5_high_real'] = data_5min[index-6]['high'] 
    current_bar['x5_low_real'] = data_5min[index-6]['low'] 
    current_bar['x5_close_real'] = data_5min[index-6]['close'] 
    current_bar['x5_height'] = current_bar['x5_high'] - current_bar['x5_low']
    current_bar['x5_body'] = max([current_bar['x5_open_real'],  current_bar['x5_close_real']]) / min([current_bar['x5_open_real'],  current_bar['x5_close_real']]) * 100
    
    current_bar['x5_roc'] = ((current_bar['x5_open_real'] / current_bar['x5_close_real']) - 1) * 100
    current_bar['x5_previous_high'] = ((current_bar['previous_high_real'] / current_bar['x5_close_real']) -1) * 100
    current_bar['x5_previous_low'] = ((current_bar['previous_low_real'] / current_bar['x5_close_real']) -1) * 100
    current_bar['x5_previous_close'] = ((current_bar['previous_close_real'] / current_bar['x5_close_real']) -1) * 100
    current_bar['x5_previous_open'] = ((current_bar['previous_open_real'] / current_bar['x5_close_real']) -1) * 100
    current_bar['x5_ema9_close'] = ((current_bar['x5_ema9_real'] / current_bar['x5_close_real']) -1) * 100
    current_bar['x5_ema21_close'] = ((current_bar['x5_ema21_real'] / current_bar['x5_close_real']) -1) * 100
    current_bar['x5_ema55_close'] = ((current_bar['x5_ema55_real'] / current_bar['x5_close_real']) -1) * 100
    current_bar['x5_ema144_close'] = ((current_bar['x5_ema144_real'] / current_bar['x5_close_real']) -1) * 100
    current_bar['x5_ema233_close'] = ((current_bar['x5_ema233_real'] / current_bar['x5_close_real']) -1) * 100
    current_bar['x5_x4_close'] = ((current_bar['x5_close_real'] / current_bar['x4_close_real']) -1) * 100
    current_bar['x5_x4_high'] = ((current_bar['x5_high_real'] / current_bar['x4_high_real']) -1) * 100
    current_bar['x5_x4_low'] = ((current_bar['x5_low_real'] / current_bar['x4_low_real']) -1) * 100        

    current_bar['x6_ema9'] = ((data_5min[index-5]['ema9'] / current_open) - 1) * 100
    current_bar['x6_ema21'] = ((data_5min[index-5]['ema21'] / current_open) - 1) * 100
    current_bar['x6_ema55'] = ((data_5min[index-5]['ema55'] / current_open) - 1) * 100
    current_bar['x6_ema144'] = ((data_5min[index-5]['ema144'] / current_open) - 1) * 100
    current_bar['x6_ema233'] = ((data_5min[index-5]['ema233'] / current_open) - 1) * 100        
    current_bar['x6_ema9_real'] = data_5min[index-5]['ema9'] 
    current_bar['x6_ema21_real'] = data_5min[index-5]['ema21']
    current_bar['x6_ema55_real'] = data_5min[index-5]['ema55']
    current_bar['x6_ema144_real'] = data_5min[index-5]['ema144']
    current_bar['x6_ema233_real'] = data_5min[index-5]['ema233']
    current_bar['x6_open'] = ((data_5min[index-5]['open'] / current_open) - 1) * 100
    current_bar['x6_high'] = ((data_5min[index-5]['high'] / current_open) - 1) * 100
    current_bar['x6_low'] = ((data_5min[index-5]['low'] / current_open) - 1) * 100
    current_bar['x6_close'] = ((data_5min[index-5]['close'] / current_open) - 1) * 100
    current_bar['x6_volume'] = ((data_5min[index-5]['volume'] / current_volume) - 1) * 100            
    current_bar['x6_open_real'] = data_5min[index-5]['open'] 
    current_bar['x6_high_real'] = data_5min[index-5]['high'] 
    current_bar['x6_low_real'] = data_5min[index-5]['low'] 
    current_bar['x6_close_real'] = data_5min[index-5]['close'] 
    current_bar['x6_height'] = current_bar['x6_high'] - current_bar['x6_low']
    current_bar['x6_body'] = max([current_bar['x6_open_real'],  current_bar['x6_close_real']]) / min([current_bar['x6_open_real'],  current_bar['x6_close_real']])
    
    current_bar['x6_roc'] = ((current_bar['x6_open_real'] / current_bar['x6_close_real']) - 1) * 100
    current_bar['x6_previous_high'] = ((current_bar['previous_high_real'] / current_bar['x6_close_real']) -1) * 100
    current_bar['x6_previous_low'] = ((current_bar['previous_low_real'] / current_bar['x6_close_real']) -1) * 100
    current_bar['x6_previous_close'] = ((current_bar['previous_close_real'] / current_bar['x6_close_real']) -1) * 100
    current_bar['x6_previous_open'] = ((current_bar['previous_open_real'] / current_bar['x6_close_real']) -1) * 100
    current_bar['x6_ema9_close'] = ((current_bar['x6_ema9_real'] / current_bar['x6_close_real']) -1) * 100
    current_bar['x6_ema21_close'] = ((current_bar['x6_ema21_real'] / current_bar['x6_close_real']) -1) * 100
    current_bar['x6_ema55_close'] = ((current_bar['x6_ema55_real'] / current_bar['x6_close_real']) -1) * 100
    current_bar['x6_ema144_close'] = ((current_bar['x6_ema144_real'] / current_bar['x6_close_real']) -1) * 100
    current_bar['x6_ema233_close'] = ((current_bar['x6_ema233_real'] / current_bar['x6_close_real']) -1) * 100
    current_bar['x6_x5_close'] = ((current_bar['x6_close_real'] / current_bar['x5_close_real']) -1) * 100
    current_bar['x6_x5_high'] = ((current_bar['x6_high_real'] / current_bar['x5_high_real']) -1) * 100
    current_bar['x6_x5_low'] = ((current_bar['x6_low_real'] / current_bar['x5_low_real']) -1) * 100        
    
    current_bar['x7_ema9'] = ((data_5min[index-4]['ema9'] / current_open) - 1) * 100
    current_bar['x7_ema21'] = ((data_5min[index-4]['ema21'] / current_open) - 1) * 100
    current_bar['x7_ema55'] = ((data_5min[index-4]['ema55'] / current_open) - 1) * 100
    current_bar['x7_ema144'] = ((data_5min[index-4]['ema144'] / current_open) - 1) * 100
    current_bar['x7_ema233'] = ((data_5min[index-4]['ema233'] / current_open) - 1) * 100        
    current_bar['x7_ema9_real'] = data_5min[index-4]['ema9'] 
    current_bar['x7_ema21_real'] = data_5min[index-4]['ema21'] 
    current_bar['x7_ema55_real'] = data_5min[index-4]['ema55'] 
    current_bar['x7_ema144_real'] = data_5min[index-4]['ema144']
    current_bar['x7_ema233_real'] = data_5min[index-4]['ema233']
    current_bar['x7_open'] = ((data_5min[index-4]['open'] / current_open) - 1) * 100
    current_bar['x7_high'] = ((data_5min[index-4]['high'] / current_open) - 1) * 100
    current_bar['x7_low'] = ((data_5min[index-4]['low'] / current_open) - 1) * 100
    current_bar['x7_close'] = ((data_5min[index-4]['close'] / current_open) - 1) * 100
    current_bar['x7_volume'] = ((data_5min[index-4]['volume'] / current_volume) - 1) * 100            
    current_bar['x7_open_real'] = data_5min[index-4]['open'] 
    current_bar['x7_high_real'] = data_5min[index-4]['high'] 
    current_bar['x7_low_real'] = data_5min[index-4]['low'] 
    current_bar['x7_close_real'] = data_5min[index-4]['close'] 
    current_bar['x7_height'] = current_bar['x7_high'] - current_bar['x7_low']
    current_bar['x7_body'] = max([current_bar['x7_open_real'],  current_bar['x7_close_real']]) / min([current_bar['x7_open_real'],  current_bar['x7_close_real']]) * 100     
    
    current_bar['x7_roc'] = ((current_bar['x7_open_real'] / current_bar['x7_close_real']) - 1) * 100
    current_bar['x7_previous_high'] = ((current_bar['previous_high_real'] / current_bar['x7_close_real']) -1) * 100
    current_bar['x7_previous_low'] = ((current_bar['previous_low_real'] / current_bar['x7_close_real']) -1) * 100
    current_bar['x7_previous_close'] = ((current_bar['previous_close_real'] / current_bar['x7_close_real']) -1) * 100
    current_bar['x7_previous_open'] = ((current_bar['previous_open_real'] / current_bar['x7_close_real']) -1) * 100
    current_bar['x7_ema9_close'] = ((current_bar['x7_ema9_real'] / current_bar['x7_close_real']) -1) * 100
    current_bar['x7_ema21_close'] = ((current_bar['x7_ema21_real'] / current_bar['x7_close_real']) -1) * 100
    current_bar['x7_ema55_close'] = ((current_bar['x7_ema55_real'] / current_bar['x7_close_real']) -1) * 100
    current_bar['x7_ema144_close'] = ((current_bar['x7_ema144_real'] / current_bar['x7_close_real']) -1) * 100
    current_bar['x7_ema233_close'] = ((current_bar['x7_ema233_real'] / current_bar['x7_close_real']) -1) * 100
    current_bar['x7_x6_close'] = ((current_bar['x7_close_real'] / current_bar['x6_close_real']) -1) * 100
    current_bar['x7_x6_high'] = ((current_bar['x7_high_real'] / current_bar['x6_high_real']) -1) * 100
    current_bar['x7_x6_low'] = ((current_bar['x7_low_real'] / current_bar['x6_low_real']) -1) * 100        
    
    current_bar['x8_ema9'] = ((data_5min[index-3]['ema9'] / current_open) - 1) * 100
    current_bar['x8_ema21'] = ((data_5min[index-3]['ema21'] / current_open) - 1) * 100
    current_bar['x8_ema55'] = ((data_5min[index-3]['ema55'] / current_open) - 1) * 100
    current_bar['x8_ema144'] = ((data_5min[index-3]['ema144'] / current_open) - 1) * 100
    current_bar['x8_ema233'] = ((data_5min[index-3]['ema233'] / current_open) - 1) * 100        
    current_bar['x8_ema9_real'] = data_5min[index-3]['ema9'] 
    current_bar['x8_ema21_real'] = data_5min[index-3]['ema21']
    current_bar['x8_ema55_real'] = data_5min[index-3]['ema55'] 
    current_bar['x8_ema144_real'] = data_5min[index-3]['ema144']
    current_bar['x8_ema233_real'] = data_5min[index-3]['ema233']
    current_bar['x8_open'] = ((data_5min[index-3]['open'] / current_open) - 1) * 100
    current_bar['x8_high'] = ((data_5min[index-3]['high'] / current_open) - 1) * 100
    current_bar['x8_low'] = ((data_5min[index-3]['low'] / current_open) - 1) * 100
    current_bar['x8_close'] = ((data_5min[index-3]['close'] / current_open) - 1) * 100
    current_bar['x8_volume'] = ((data_5min[index-3]['volume'] / current_volume) - 1) * 100            
    current_bar['x8_open_real'] = data_5min[index-3]['open'] 
    current_bar['x8_high_real'] = data_5min[index-3]['high'] 
    current_bar['x8_low_real'] = data_5min[index-3]['low'] 
    current_bar['x8_close_real'] = data_5min[index-3]['close']    
    current_bar['x8_height'] = current_bar['x8_high'] - current_bar['x8_low']
    current_bar['x8_body'] = max([current_bar['x8_open_real'],  current_bar['x8_close_real']]) / min([current_bar['x8_open_real'],  current_bar['x8_close_real']]) * 100     
    
    current_bar['x8_roc'] = ((current_bar['x8_open_real'] / current_bar['x8_close_real']) - 1) * 100
    current_bar['x8_previous_high'] = ((current_bar['previous_high_real'] / current_bar['x8_close_real']) -1) * 100
    current_bar['x8_previous_low'] = ((current_bar['previous_low_real'] / current_bar['x8_close_real']) -1) * 100
    current_bar['x8_previous_close'] = ((current_bar['previous_close_real'] / current_bar['x8_close_real']) -1) * 100
    current_bar['x8_previous_open'] = ((current_bar['previous_open_real'] / current_bar['x8_close_real']) -1) * 100
    current_bar['x8_ema9_close'] = ((current_bar['x8_ema9_real'] / current_bar['x8_close_real']) -1) * 100
    current_bar['x8_ema21_close'] = ((current_bar['x8_ema21_real'] / current_bar['x8_close_real']) -1) * 100
    current_bar['x8_ema55_close'] = ((current_bar['x8_ema55_real'] / current_bar['x8_close_real']) -1) * 100
    current_bar['x8_ema144_close'] = ((current_bar['x8_ema144_real'] / current_bar['x8_close_real']) -1) * 100
    current_bar['x8_ema233_close'] = ((current_bar['x8_ema233_real'] / current_bar['x8_close_real']) -1) * 100
    current_bar['x8_x7_close'] = ((current_bar['x8_close_real'] / current_bar['x7_close_real']) -1) * 100
    current_bar['x8_x7_high'] = ((current_bar['x8_high_real'] / current_bar['x7_high_real']) -1) * 100
    current_bar['x8_x7_low'] = ((current_bar['x8_low_real'] / current_bar['x7_low_real']) -1) * 100        
    
    current_bar['x9_ema9'] = ((data_5min[index-2]['ema9'] / current_open) - 1) * 100
    current_bar['x9_ema21'] = ((data_5min[index-2]['ema21'] / current_open) - 1) * 100
    current_bar['x9_ema55'] = ((data_5min[index-2]['ema55'] / current_open) - 1) * 100
    current_bar['x9_ema144'] = ((data_5min[index-2]['ema144'] / current_open) - 1) * 100
    current_bar['x9_ema233'] = ((data_5min[index-2]['ema233'] / current_open) - 1) * 100        
    current_bar['x9_ema9_real'] = data_5min[index-2]['ema9'] 
    current_bar['x9_ema21_real'] = data_5min[index-2]['ema21'] 
    current_bar['x9_ema55_real'] = data_5min[index-2]['ema55'] 
    current_bar['x9_ema144_real'] = data_5min[index-2]['ema144']
    current_bar['x9_ema233_real'] = data_5min[index-2]['ema233']
    current_bar['x9_open'] = ((data_5min[index-2]['open'] / current_open) - 1) * 100
    current_bar['x9_high'] = ((data_5min[index-2]['high'] / current_open) - 1) * 100
    current_bar['x9_low'] = ((data_5min[index-2]['low'] / current_open) - 1) * 100
    current_bar['x9_close'] = ((data_5min[index-2]['close'] / current_open) - 1) * 100
    current_bar['x9_volume'] = ((data_5min[index-2]['volume'] / current_volume) - 1) * 100            
    current_bar['x9_open_real'] = data_5min[index-2]['open'] 
    current_bar['x9_high_real'] = data_5min[index-2]['high'] 
    current_bar['x9_low_real'] = data_5min[index-2]['low'] 
    current_bar['x9_close_real'] = data_5min[index-2]['close']
    current_bar['x9_height'] = current_bar['x9_high'] - current_bar['x9_low']
    current_bar['x9_body'] = max([current_bar['x9_open_real'],  current_bar['x9_close_real']]) / min([current_bar['x9_open_real'],  current_bar['x9_close_real']]) * 100         
    
    current_bar['x9_roc'] = ((current_bar['x9_open_real'] / current_bar['x9_close_real']) - 1) * 100
    current_bar['x9_previous_high'] = ((current_bar['previous_high_real'] / current_bar['x9_close_real']) -1) * 100
    current_bar['x9_previous_low'] = ((current_bar['previous_low_real'] / current_bar['x9_close_real']) -1) * 100
    current_bar['x9_previous_close'] = ((current_bar['previous_close_real'] / current_bar['x9_close_real']) -1) * 100
    current_bar['x9_previous_open'] = ((current_bar['previous_open_real'] / current_bar['x9_close_real']) -1) * 100
    current_bar['x9_ema9_close'] = ((current_bar['x9_ema9_real'] / current_bar['x9_close_real']) -1) * 100
    current_bar['x9_ema21_close'] = ((current_bar['x9_ema21_real'] / current_bar['x9_close_real']) -1) * 100
    current_bar['x9_ema55_close'] = ((current_bar['x9_ema55_real'] / current_bar['x9_close_real']) -1) * 100
    current_bar['x9_ema144_close'] = ((current_bar['x9_ema144_real'] / current_bar['x9_close_real']) -1) * 100
    current_bar['x9_ema233_close'] = ((current_bar['x9_ema233_real'] / current_bar['x9_close_real']) -1) * 100
    current_bar['x9_x8_close'] = ((current_bar['x9_close_real'] / current_bar['x8_close_real']) -1) * 100
    current_bar['x9_x8_high'] = ((current_bar['x9_high_real'] / current_bar['x8_high_real']) -1) * 100
    current_bar['x9_x8_low'] = ((current_bar['x9_low_real'] / current_bar['x8_low_real']) -1) * 100        
    
    current_bar['x10_ema9'] = ((data_5min[index-1]['ema9'] / current_open) - 1) * 100
    current_bar['x10_ema21'] = ((data_5min[index-1]['ema21'] / current_open) - 1) * 100
    current_bar['x10_ema55'] = ((data_5min[index-1]['ema55'] / current_open) - 1) * 100
    current_bar['x10_ema144'] = ((data_5min[index-1]['ema144'] / current_open) - 1) * 100
    current_bar['x10_ema233'] = ((data_5min[index-1]['ema233'] / current_open) - 1) * 100    
    current_bar['x10_ema9_real'] = data_5min[index-1]['ema9'] 
    current_bar['x10_ema21_real'] = data_5min[index-1]['ema21']
    current_bar['x10_ema55_real'] = data_5min[index-1]['ema55']
    current_bar['x10_ema144_real'] = data_5min[index-1]['ema144']
    current_bar['x10_ema233_real'] = data_5min[index-1]['ema233']
    current_bar['x10_open'] = ((data_5min[index-1]['open'] / current_open) - 1) * 100
    current_bar['x10_high'] = ((data_5min[index-1]['high'] / current_open) - 1) * 100
    current_bar['x10_low'] = ((data_5min[index-1]['low'] / current_open) - 1) * 100
    current_bar['x10_close'] = ((data_5min[index-1]['close'] / current_open) - 1) * 100
    current_bar['x10_volume'] = ((data_5min[index-1]['volume'] / current_volume) - 1) * 100            
    current_bar['x10_open_real'] = data_5min[index-1]['open'] 
    current_bar['x10_high_real'] = data_5min[index-1]['high'] 
    current_bar['x10_low_real'] = data_5min[index-1]['low'] 
    current_bar['x10_close_real'] = data_5min[index-1]['close']   
    current_bar['x10_height'] = current_bar['x10_high'] - current_bar['x10_low']
    current_bar['x10_body'] = max([current_bar['x10_open_real'],  current_bar['x10_close_real']]) / min([current_bar['x10_open_real'],  current_bar['x10_close_real']]) * 100     
    
    current_bar['x10_roc'] = ((current_bar['x10_open_real'] / current_bar['x10_close_real']) - 1) * 100
    current_bar['x10_previous_high'] = ((current_bar['previous_high_real'] / current_bar['x10_close_real']) -1) * 100
    current_bar['x10_previous_low'] = ((current_bar['previous_low_real'] / current_bar['x10_close_real']) -1) * 100
    current_bar['x10_previous_close'] = ((current_bar['previous_close_real'] / current_bar['x10_close_real']) -1) * 100
    current_bar['x10_previous_open'] = ((current_bar['previous_open_real'] / current_bar['x10_close_real']) -1) * 100
    current_bar['x10_ema9_close'] = ((current_bar['x10_ema9_real'] / current_bar['x10_close_real']) -1) * 100
    current_bar['x10_ema21_close'] = ((current_bar['x10_ema21_real'] / current_bar['x10_close_real']) -1) * 100
    current_bar['x10_ema55_close'] = ((current_bar['x10_ema55_real'] / current_bar['x10_close_real']) -1) * 100
    current_bar['x10_ema144_close'] = ((current_bar['x10_ema144_real'] / current_bar['x10_close_real']) -1) * 100
    current_bar['x10_ema233_close'] = ((current_bar['x10_ema233_real'] / current_bar['x10_close_real']) -1) * 100
    current_bar['x10_x9_close'] = ((current_bar['x10_close_real'] / current_bar['x9_close_real']) -1) * 100
    current_bar['x10_x9_high'] = ((current_bar['x10_high_real'] / current_bar['x9_high_real']) -1) * 100
    current_bar['x10_x9_low'] = ((current_bar['x10_low_real'] / current_bar['x9_low_real']) -1) * 100        
    
    
    
    current_bar['x11_ema9'] = ((data_5min[index]['ema9'] / current_open) - 1) * 100
    current_bar['x11_ema21'] = ((data_5min[index]['ema21'] / current_open) - 1) * 100
    current_bar['x11_ema55'] = ((data_5min[index]['ema55'] / current_open) - 1) * 100
    current_bar['x11_ema144'] = ((data_5min[index]['ema144'] / current_open) - 1) * 100
    current_bar['x11_ema233'] = ((data_5min[index]['ema233'] / current_open) - 1) * 100        
    current_bar['x11_ema9_real'] = data_5min[index]['ema9'] 
    current_bar['x11_ema21_real'] = data_5min[index]['ema21'] 
    current_bar['x11_ema55_real'] = data_5min[index]['ema55'] 
    current_bar['x11_ema144_real'] = data_5min[index]['ema144']
    current_bar['x11_ema233_real'] = data_5min[index]['ema233']
    current_bar['x11_open'] = ((data_5min[index]['open'] / current_open) - 1) * 100
    current_bar['x11_high'] = ((data_5min[index]['high'] / current_open) - 1) * 100
    current_bar['x11_low'] = ((data_5min[index]['low'] / current_open) - 1) * 100
    current_bar['x11_close'] = ((data_5min[index]['close'] / current_open) - 1) * 100
    current_bar['x11_volume'] = ((data_5min[index]['volume'] / current_volume) - 1) * 100            
    current_bar['x11_open_real'] = data_5min[index]['open'] 
    current_bar['x11_high_real'] = data_5min[index]['high'] 
    current_bar['x11_low_real'] = data_5min[index]['low'] 
    current_bar['x11_close_real'] = data_5min[index]['close']    
    current_bar['x11_height'] = current_bar['x11_high'] - current_bar['x11_low']
    current_bar['x11_body'] = max([current_bar['x11_open_real'],  current_bar['x11_close_real']]) / min([current_bar['x11_open_real'],  current_bar['x11_close_real']]) * 100     
    
    current_bar['x11_roc'] = ((current_bar['x11_open_real'] / current_bar['x11_close_real']) - 1) * 100
    current_bar['x11_previous_high'] = ((current_bar['previous_high_real'] / current_bar['x11_close_real']) -1) * 100
    current_bar['x11_previous_low'] = ((current_bar['previous_low_real'] / current_bar['x11_close_real']) -1) * 100
    current_bar['x11_previous_close'] = ((current_bar['previous_close_real'] / current_bar['x11_close_real']) -1) * 100
    current_bar['x11_previous_open'] = ((current_bar['previous_open_real'] / current_bar['x11_close_real']) -1) * 100
    current_bar['x11_ema9_close'] = ((current_bar['x11_ema9_real'] / current_bar['x11_close_real']) -1) * 100
    current_bar['x11_ema21_close'] = ((current_bar['x11_ema21_real'] / current_bar['x11_close_real']) -1) * 100
    current_bar['x11_ema55_close'] = ((current_bar['x11_ema55_real'] / current_bar['x11_close_real']) -1) * 100
    current_bar['x11_ema144_close'] = ((current_bar['x11_ema144_real'] / current_bar['x11_close_real']) -1) * 100
    current_bar['x11_ema233_close'] = ((current_bar['x11_ema233_real'] / current_bar['x11_close_real']) -1) * 100
    current_bar['x11_x10_close'] = ((current_bar['x11_close_real'] / current_bar['x10_close_real']) -1) * 100
    current_bar['x11_x10_high'] = ((current_bar['x11_high_real'] / current_bar['x10_high_real']) -1) * 100
    current_bar['x11_x10_low'] = ((current_bar['x11_low_real'] / current_bar['x10_low_real']) -1) * 100        
    
    current_bar['x11_high_slope'] = get_slope(current_bar,'high')
    current_bar['x11_low_slope'] = get_slope(current_bar,'low')
    current_bar['x11_close_slope'] = get_slope(current_bar,'low')
    current_bar['x11_volume_slope'] = get_slope(current_bar,'volume')
    
    current_bar['chart_data'] = [
        {
        'date': 0,
        'open': current_bar['x0_open'],
        'high': current_bar['x0_high'],
        'low': current_bar['x0_low'],
        'close': current_bar['x0_close'],
        'ema9': current_bar['x0_ema9'],
        'ema21': current_bar['x0_ema21'],
        'open_real': current_bar['x0_open_real'],
        'high_real': current_bar['x0_high_real'],
        'low_real': current_bar['x0_low_real'],
        'close_real': current_bar['x0_close_real']
        },
        {
        'date': 1,
        'open': current_bar['x1_open'],
        'high': current_bar['x1_high'],
        'low': current_bar['x1_low'],
        'close': current_bar['x1_close'],
        'ema9': current_bar['x1_ema9'],
        'ema21': current_bar['x1_ema21'],
        'open_real': current_bar['x1_open_real'],
        'high_real': current_bar['x1_high_real'],
        'low_real': current_bar['x1_low_real'],
        'close_real': current_bar['x1_close_real']
        },
        {
        'date': 2,
        'open': current_bar['x2_open'],
        'high': current_bar['x2_high'],
        'low': current_bar['x2_low'],
        'close': current_bar['x2_close'],
        'ema9': current_bar['x2_ema9'],
        'ema21': current_bar['x2_ema21'],
        'open_real': current_bar['x2_open_real'],
        'high_real': current_bar['x2_high_real'],
        'low_real': current_bar['x2_low_real'],
        'close_real': current_bar['x2_close_real']
        },
        {
        'date': 3,
        'open': current_bar['x3_open'],
        'high': current_bar['x3_high'],
        'low': current_bar['x3_low'],
        'close': current_bar['x3_close'],
        'ema9': current_bar['x3_ema9'],
        'ema21': current_bar['x3_ema21'],
        'open_real': current_bar['x3_open_real'],
        'high_real': current_bar['x3_high_real'],
        'low_real': current_bar['x3_low_real'],
        'close_real': current_bar['x3_close_real']
        },     
        {
        'date': 4,
        'open': current_bar['x4_open'],
        'high': current_bar['x4_high'],
        'low': current_bar['x4_low'],
        'close': current_bar['x4_close'],
        'ema9': current_bar['x4_ema9'],
        'ema21': current_bar['x4_ema21'],
        'open_real': current_bar['x4_open_real'],
        'high_real': current_bar['x4_high_real'],
        'low_real': current_bar['x4_low_real'],
        'close_real': current_bar['x4_close_real']
        },             
        {
        'date': 5,
        'open': current_bar['x5_open'],
        'high': current_bar['x5_high'],
        'low': current_bar['x5_low'],
        'close': current_bar['x5_close'],
        'ema9': current_bar['x5_ema9'],
        'ema21': current_bar['x5_ema21'],
        'open_real': current_bar['x5_open_real'],
        'high_real': current_bar['x5_high_real'],
        'low_real': current_bar['x5_low_real'],
        'close_real': current_bar['x5_close_real']
        },
        {
        'date': 6,
        'open': current_bar['x6_open'],
        'high': current_bar['x6_high'],
        'low': current_bar['x6_low'],
        'close': current_bar['x6_close'],
        'ema9': current_bar['x6_ema9'],
        'ema21': current_bar['x6_ema21'],
        'open_real': current_bar['x6_open_real'],
        'high_real': current_bar['x6_high_real'],
        'low_real': current_bar['x6_low_real'],
        'close_real': current_bar['x6_close_real']
        },
        {
        'date': 7,
        'open': current_bar['x7_open'],
        'high': current_bar['x7_high'],
        'low': current_bar['x7_low'],
        'close': current_bar['x7_close'],
        'ema9': current_bar['x7_ema9'],
        'ema21': current_bar['x7_ema21'],
        'open_real': current_bar['x7_open_real'],
        'high_real': current_bar['x7_high_real'],
        'low_real': current_bar['x7_low_real'],
        'close_real': current_bar['x7_close_real']
        },
        {
        'date': 8,
        'open': current_bar['x8_open'],
        'high': current_bar['x8_high'],
        'low': current_bar['x8_low'],
        'close': current_bar['x8_close'],
        'ema9': current_bar['x8_ema9'],
        'ema21': current_bar['x8_ema21'],
        'open_real': current_bar['x8_open_real'],
        'high_real': current_bar['x8_high_real'],
        'low_real': current_bar['x8_low_real'],
        'close_real': current_bar['x8_close_real']
        },
        {
        'date': 9,
        'open': current_bar['x9_open'],
        'high': current_bar['x9_high'],
        'low': current_bar['x9_low'],
        'close': current_bar['x9_close'],
        'ema9': current_bar['x9_ema9'],
        'ema21': current_bar['x9_ema21'],
        'open_real': current_bar['x9_open_real'],
        'high_real': current_bar['x9_high_real'],
        'low_real': current_bar['x9_low_real'],
        'close_real': current_bar['x9_close_real']
        },        
        {
        'date': 10,
        'open': current_bar['x10_open'],
        'high': current_bar['x10_high'],
        'low': current_bar['x10_low'],
        'close': current_bar['x10_close'],
        'ema9': current_bar['x10_ema9'],
        'ema21': current_bar['x10_ema21'],
        'open_real': current_bar['x10_open_real'],
        'high_real': current_bar['x10_high_real'],
        'low_real': current_bar['x10_low_real'],
        'close_real': current_bar['x10_close_real']
        },        
        {
        'date': 11,
        'open': current_bar['x11_open'],
        'high': current_bar['x11_high'],
        'low': current_bar['x11_low'],
        'close': current_bar['x11_close'],
        'ema9': current_bar['x11_ema9'],
        'ema21': current_bar['x11_ema21'],
        'open_real': current_bar['x11_open_real'],
        'high_real': current_bar['x11_high_real'],
        'low_real': current_bar['x11_low_real'],
        'close_real': current_bar['x11_close_real']
        },                
    ]
    
    current_bar['current_pos'] = current_bar['x11_close']
    bars_for_evaluation = [x for x in current_bar['chart_data'] if x['date'] >= 12] 
    current_bar['current_date'] = current_date
    
    generated_bars.append(current_bar)
    
generated_bars_count = len(generated_bars)

In [ ]:
test_generated_bars = pd.DataFrame(generated_bars).head(10)
test_generated_bars

In [ ]:
if True:
    TOOLS = "pan,wheel_zoom,box_zoom,reset,save"
    generated_bars_to_visualize = generated_bars[1]
    df = pd.DataFrame(generated_bars_to_visualize['chart_data'])

    df['previous_high']=generated_bars_to_visualize['previous_high']
    df['previous_low']=generated_bars_to_visualize['previous_low']
    df['previous_close']=generated_bars_to_visualize['previous_close']
    df['previous_open']=generated_bars_to_visualize['previous_open']

    df['previous_high_real']=generated_bars_to_visualize['previous_high_real']
    df['previous_low_real']=generated_bars_to_visualize['previous_low_real']
    df['previous_close_real']=generated_bars_to_visualize['previous_close_real']
    df['previous_open_real']=generated_bars_to_visualize['previous_open_real']
    df['current_pos']=generated_bars_to_visualize['current_pos']
    
    current_start = generated_bars_to_visualize['x5_close_real']
    current_date = generated_bars_to_visualize['current_date']
    current_time = generated_bars_to_visualize['current_time']

    p = figure(x_axis_type="linear", tools=TOOLS, width=1000, title = f'WDO Candlestick, start:{current_start}, date:{current_date}, time:{current_time}')
    p.xaxis.major_label_orientation = pi/4
    p.grid.grid_line_alpha=0.3
    w = 12*60*60*1000 # half day in ms

    inc = df.close > df.open
    dec = df.open > df.close

    p.segment(df.date, df.high, df.date, df.low, color="black")
    p.vbar(df.date[inc], 0.8, df.open[inc], df.close[inc], fill_color="#D5E1DD", line_color="black")
    p.vbar(df.date[dec], 0.8, df.open[dec], df.close[dec], fill_color="#F2583E", line_color="black")

    p.line(df.date, df.ema9, line_color="#00FF00", line_width = 2)
    p.line(df.date, df.ema21, line_color="#FFFF00", line_width = 2)
    p.line(df.date, df.previous_high, line_color="#FF0000", line_width = 2)
    p.line(df.date, df.previous_close, line_color="#FF0000", line_width = 2)
    p.line(df.date, df.previous_open, line_color="#FF0000", line_width = 2)
    p.line(df.date, df.previous_low, line_color="#FF0000", line_width = 2)
    p.line(df.date, df.current_pos, line_color="#000000", line_width = 2)

    show(p)

In [ ]:
best_short_booster = joblib.load(models_found[first_model]['xgboostshortmodel'])
best_long_booster = joblib.load(models_found[first_model]['xgboostlongmodel'])

In [ ]:
df_current_total_dataset = pd.DataFrame(generated_bars)
current_raw_dataset = generated_bars_raw

In [ ]:
def predict_short(row):
    a = row[current_parameters['CURRENT_X_COLUMNS']].to_numpy().reshape(1,-1)
    return best_short_booster.inplace_predict(a)[0]

def predict_long(row):
    a = row[current_parameters['CURRENT_X_COLUMNS']].to_numpy().reshape(1,-1)
    return best_long_booster.inplace_predict(a)[0]

df_current_total_dataset['short_predict'] = df_current_total_dataset.apply( lambda row: predict_short(row), axis=1)
df_current_total_dataset['long_predict'] = df_current_total_dataset.apply( lambda row: predict_long(row), axis=1)

In [ ]:
def predict_shap_short(row):
    x_columns = row[current_parameters['CURRENT_X_COLUMNS']].to_numpy().reshape(1,-1)
    explainer = shap.TreeExplainer(best_short_booster)
    shap_values = explainer.shap_values(x_columns)
    shap_values = shap_values[0]
    shap_values_with_desc = []
    for current_column in range(len(current_parameters['CURRENT_X_COLUMNS'])):
        shap_values_with_desc.append({ 'desc':current_parameters['CURRENT_X_COLUMNS'][current_column], 'value':shap_values[current_column]})

    shap_values_with_desc.sort(key=lambda x: x['value'])
    return str(shap_values_with_desc)
    

def predict_shap_long(row):
    x_columns = row[current_parameters['CURRENT_X_COLUMNS']].to_numpy().reshape(1,-1)
    explainer = shap.TreeExplainer(best_long_booster)
    shap_values = explainer.shap_values(x_columns)
    shap_values = shap_values[0]
    shap_values_with_desc = []
    for current_column in range(len(current_parameters['CURRENT_X_COLUMNS'])):
        shap_values_with_desc.append({ 'desc':current_parameters['CURRENT_X_COLUMNS'][current_column], 'value':shap_values[current_column]})

    shap_values_with_desc.sort(key=lambda x: x['value'])
    return str(shap_values_with_desc)

    
df_current_total_dataset['short_shap'] = df_current_total_dataset.apply( lambda row: predict_shap_short(row) if (row['short_predict'] > 0 and False) else "", axis=1)
df_current_total_dataset['long_shap'] = df_current_total_dataset.apply( lambda row: predict_shap_long(row) if (row['long_predict'] > 0 and False) else "", axis=1)

In [ ]:
df_current_total_dataset['short_predict'].hist(bins=500)

In [ ]:
df_current_total_dataset['long_predict'].hist(bins=500)

In [ ]:
results_from_df = df_current_total_dataset.to_dict('records')
results = {}
short_results = []
long_results = []

for current_result in results_from_df:
    results[(current_result['current_date'] * 10000) + current_result['current_time']] = \
    { 
        'short_predict' : current_result['short_predict'],
        'long_predict' : current_result['long_predict'],
        'short_shap' : current_result['short_shap'],
        'long_shap' : current_result['long_shap'],
    }    
    short_results.append(current_result['short_predict'])
    long_results.append(current_result['long_predict'])


In [ ]:
minimum_short_predict = 0
minimum_long_predict = 0

maximum_short_predict = int(round(max(short_results),0))
maximum_long_predict = int(round(max(long_results),0))

current_short_predict = 0
current_long_predict = 0

current_trade = None
current_trade_entries = []

candle_count = 0
current_target = current_parameters['CURRENT_TARGET']
current_stop = current_parameters['CURRENT_STOP']

for current_candle in current_raw_dataset:
    if current_trade is not None:
        is_short = current_trade['trade_type'] == 'short'
        is_long = current_trade['trade_type'] == 'long'
        if is_short:
            if current_candle['low'] <= current_trade['trade_target']:
                current_trade['result'] =  round(current_trade['trade_start'] - current_trade['trade_target'],2)
                current_trade['final_close'] = current_candle['close']
                current_trade = None
                continue
            if current_candle['high'] >= current_trade['trade_stop']:
                current_trade['result'] =  round(current_trade['trade_start'] - current_trade['trade_stop'],2)
                current_trade['final_close'] = current_candle['close']
                current_trade = None
                continue                
        if is_long:
            if current_candle['low'] <= current_trade['trade_stop']:
                current_trade['result'] =  round(current_candle['close'] - current_trade['trade_stop'],2)
                current_trade['final_close'] = current_candle['close']
                current_trade = None
                continue
            if current_candle['high'] >= current_trade['trade_target']:
                current_trade['result'] =  round(current_candle['close'] - current_trade['trade_target'],2)
                current_trade['final_close'] = current_candle['close']
                current_trade = None
                continue                
        if candle_count > (current_trade['start_candle'] + MAX_TRADE_DURATION):
            if is_long:
                current_trade['result'] =  round(current_candle['close'] - current_trade['trade_start'],2)
                current_trade['final_close'] = current_candle['close']
            if is_short:
                current_trade['result'] =  round(current_trade['trade_start'] - current_candle['close'],2)
                current_trade['final_close'] = current_candle['close']
            current_trade = None
            continue
        
    if current_trade is None:
        if current_candle['Date'] > TEST_MINIMUM_DATE_TRADE and \
           current_candle['Time'] >= current_parameters['MINIMUM_TIME'] and \
           current_candle['Time'] <= current_parameters['MAXIMUM_TIME']:
            current_date_time = (current_candle['Date'] * 10000) + current_candle['Time']
            is_entry_point = results[current_date_time]['short_predict'] > DECISION_BOUNDARY or results[current_date_time]['long_predict'] > DECISION_BOUNDARY
            if is_entry_point:
                is_short = results[current_date_time]['short_predict'] > 0 
                current_trade = {**current_candle, 
                                 'trade_type': 'short' if is_short else 'long', 
                                 'start_candle': candle_count,
                                 'trade_start': current_candle['close'],
                                 'predicted': results[current_date_time]['short_predict'] if is_short else results[current_date_time]['long_predict'],
                                 'shap': results[current_date_time]['short_shap'] if is_short else results[current_date_time]['long_shap']
                                }
                if is_short:
                    current_trade['trade_target'] = current_candle['close'] * (1 - (current_stop / 100))
                    current_trade['trade_stop'] = current_candle['close'] * (1 + (current_target / 100))
                else:
                    current_trade['trade_target'] = current_candle['close'] * (1 + (current_target / 100))
                    current_trade['trade_stop'] = current_candle['close'] * (1 - (current_stop / 100))
                    
                current_trade_entries.append(current_trade)
                
    candle_count += 1

In [ ]:
trades = pd.DataFrame(current_trade_entries)
raw_trades_file_name = os.path.join(f"{CURRENT_ASSET}.{CURRENT_TIMEFRAME}.{int(CURRENT_TARGET * 100)}.{int(CURRENT_STOP * 100)}.test_trades.xlsx")
raw_trades_full_file_name = os.path.join('..','00.data','output',raw_trades_file_name)
trades.to_excel(raw_trades_full_file_name)
trades

In [ ]:
current_trade_per_day = {}
days = []
current_days = []
current_results = []
for current_trade in current_trade_entries:
    current_date = current_trade['Date']        
    if current_date in current_trade_per_day:
        current_trade_per_day[current_date]['result'] += current_trade['result']
    else:
        current_trade_per_day[current_date] = {
            'result' : current_trade['result']
        }
        
for current_date in current_trade_per_day:
    days.append({'date': current_date, 'result': current_trade_per_day[current_date]['result']})
    current_days.append(str(current_date))
    current_results.append(current_trade_per_day[current_date]['result'])
   
current_source = ColumnDataSource(data=dict(current_days=current_days, current_results=current_results))
p = figure(x_range=current_days, width=800, height=450, toolbar_location=None, title="Resultado do Modelo Em pontos")
p.vbar(x='current_days', top='current_results', width=0.9, source=current_source)

show(p)